### exp004 - fine-tuning LLaMA3.1 8b instruct w/ LoRA (PEFT) via ~3057 labeled headlines
https://colab.research.google.com/drive/1m5--orgqdnCUtSKUGH4WR1CZ_UB7FkPP?usp=sharing

In [2]:
import pandas as pd
df = pd.read_csv("../../results/exp004/results.csv")

In [3]:
from sklearn.metrics import confusion_matrix
'''
[[TP FN]
 [FP TN]]
'''
cm = confusion_matrix(df["Direção"], df["Prediction"], labels=["Aumento", "Diminuição"])
print(cm)

[[129  98]
 [102  71]]


In [4]:
from sklearn.metrics import classification_report

# Normalize predictions (just in case)
df["Prediction"] = df["Prediction"].str.strip().str.capitalize()
df["Direção"] = df["Direção"].str.strip().str.capitalize()

# Report
report = classification_report(
    df["Direção"], df["Prediction"],
    labels=["Aumento", "Diminuição"],
    target_names=["Aumento", "Diminuição"],
    digits=3
)

print("\nClassification Report:\n")
print(report)


Classification Report:

              precision    recall  f1-score   support

     Aumento      0.558     0.568     0.563       227
  Diminuição      0.420     0.410     0.415       173

    accuracy                          0.500       400
   macro avg      0.489     0.489     0.489       400
weighted avg      0.499     0.500     0.499       400

